In [46]:
import os
from os import listdir
from os.path import isfile, join, exists

In [47]:
directory = "Dataset2/"
categories_name = os.listdir(directory)

In [48]:
print(categories_name)

['airport_terminal', 'art_gallery', 'art_studio', 'childs_room']


In [54]:
def file_path(path):
    file = [path+f for f in listdir(path) if isfile(join(path, f))]
    return file

In [55]:
labels=[]
file = []
for cat in categories_name:
    files  = file_path(directory + cat +"/")
    for f in files:
        file.append(f)
        labels.append(categories_name.index(cat))
    

In [57]:
len(file)

317